In [1]:
with open(file='/kaggle/input/moby-dick-herman-melville/melville-moby_dick.txt', mode='r', encoding='utf-8', ) as input_fp:
    data = input_fp.read()

In [2]:
from arrow import now
from spacy import load
time_start = now()
spacy_model = load('en_core_web_sm')
spacy_model.max_length = 1200000
spacy_result = spacy_model(data.replace('\n', ' '))
print(now() - time_start)

0:00:47.624911


In [3]:
documents = [item.text for item in spacy_result.sents]

In [4]:
from pprint import pprint  # pretty-printer
from collections import defaultdict
from gensim.parsing.preprocessing import preprocess_documents

# # remove common words and tokenize
# stoplist = set('for a of the and to in'.split())
# texts = [
#     [word for word in document.lower().split() if word not in stoplist]
#     for document in documents
# ]
texts = preprocess_documents(docs=documents)

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

In [5]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
dictionary.save('/kaggle/working/melville.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary<6600 unique tokens: ['chapter', 'loom', 'ishmael', 'ago', 'have']...>


TODO: find a better way to preprocess this, so we don't have so much punctuation in our corpus.

In [6]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [7]:
from gensim.models import TfidfModel

tfidf = TfidfModel(corpus)  # step 1 -- initialize a model

In [8]:
from gensim.models import HdpModel
model = HdpModel(corpus, id2word=dictionary)

In [9]:
model.print_topics()

[(0,
  '0.002*whale + 0.001*gener + 0.001*ship + 0.001*sea + 0.001*time + 0.001*nantucket + 0.001*ey + 0.001*duti + 0.001*envi + 0.001*gardin'),
 (1,
  '0.002*boat + 0.001*whale + 0.001*extent + 0.001*digniti + 0.001*arm + 0.001*like + 0.001*said + 0.001*bank + 0.001*aromat + 0.001*latitud'),
 (2,
  '0.002*whale + 0.001*cob + 0.001*shoreless + 0.001*interest + 0.001*treat + 0.001*supremaci + 0.001*rampart + 0.001*bliss + 0.001*reev + 0.001*poniard'),
 (3,
  '0.002*whale + 0.002*great + 0.001*claim + 0.001*yea + 0.001*like + 0.001*hosea + 0.001*greatest + 0.001*brim + 0.001*boat + 0.001*wear'),
 (4,
  '0.001*ahab + 0.001*dai + 0.001*ey + 0.001*head + 0.001*think + 0.001*sentri + 0.001*perceiv + 0.001*eloqu + 0.001*whale + 0.001*grimli'),
 (5,
  '0.001*beard + 0.001*ebb + 0.001*stubb + 0.001*concert + 0.001*beneath + 0.001*whale + 0.001*prometheu + 0.001*africa + 0.001*men + 0.001*round'),
 (6,
  '0.002*often + 0.001*whale + 0.001*gasp + 0.001*right + 0.001*launch + 0.001*affair + 0.001*